In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import pickle
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Books Embeddings

In [3]:
book_metadata_df = pd.read_json('/content/drive/MyDrive/Priyanka/final_book_metadata.json', lines=True)

In [4]:
book_metadata_df.head()

,category,description,title,brand,price,asin
0,"[Books, Literature & Fiction, Dramas & Plays]",[William Shakespeare is widely regarded as the...,Love's Labour's Lost: Performed by Derek Jacob...,Visit Amazon's William Shakespeare Page,$20.93,0001050230
1,"[Books, New, Used & Rental Textbooks, Humanities]",[William Shakespeare is widely regarded as the...,Othello: Complete &amp; Unabridged,Visit Amazon's William Shakespeare Page,,0001048767
2,"[Books, Children's Books, Literature & Fiction]",[],The Secret Garden,Frances Hodgson [illustrated by ruth sanderson...,$4.72,0001945424
3,"[Books, Literature & Fiction, Literary]",[],Ten Little Niggers,Visit Amazon's Agatha Christie Page,$50.63,0002318350
4,"[Books, Mystery, Thriller &amp; Suspense, Thri...",[Jan Needle is a well known children's writer....,Dracula (Collins Drama),Jan Needle,,0003302245


In [5]:
book_metadata_df['category'] = book_metadata_df['category'].apply(' '.join)
book_metadata_df['description'] = book_metadata_df['description'].apply(' '.join)

book_metadata_df['category'] = book_metadata_df['category'].str.replace('Books', '', case=False)
#book_metadata_df['brand'] = book_metadata_df['brand'].str.replace('Visit Amazon\'s', '', case=False)
#book_metadata_df['brand'] = book_metadata_df['brand'].str.replace('Page', '', case=False)

In [6]:
book_metadata_df.head()

,category,description,title,brand,price,asin
0,Literature & Fiction Dramas & Plays,William Shakespeare is widely regarded as the ...,Love's Labour's Lost: Performed by Derek Jacob...,Visit Amazon's William Shakespeare Page,$20.93,0001050230
1,"New, Used & Rental Text Humanities",William Shakespeare is widely regarded as the ...,Othello: Complete &amp; Unabridged,Visit Amazon's William Shakespeare Page,,0001048767
2,Children's Literature & Fiction,,The Secret Garden,Frances Hodgson [illustrated by ruth sanderson...,$4.72,0001945424
3,Literature & Fiction Literary,,Ten Little Niggers,Visit Amazon's Agatha Christie Page,$50.63,0002318350
4,"Mystery, Thriller &amp; Suspense Thrillers &a...",Jan Needle is a well known children's writer. ...,Dracula (Collins Drama),Jan Needle,,0003302245


In [7]:
def pre_process_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers and special characters
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    return text

In [8]:
book_metadata_df['category'] = book_metadata_df['category'].apply(pre_process_text)
book_metadata_df['description'] = book_metadata_df['description'].apply(pre_process_text)
book_metadata_df['title'] = book_metadata_df['title'].apply(pre_process_text)
#book_metadata_df['brand'] = book_metadata_df['brand'].apply(pre_process_text)

In [9]:
book_metadata_df['book_data'] = book_metadata_df['category'] + " " + book_metadata_df['title'] + " " + book_metadata_df['description']

In [10]:
book_metadata_df.head()

,category,description,title,brand,price,asin,book_data
0,literature fiction dramas plays,william shakespeare is widely regarded as the ...,loves labours lost performed by derek jacobi g...,Visit Amazon's William Shakespeare Page,$20.93,0001050230,literature fiction dramas plays loves labou...
1,new used rental text humanities,william shakespeare is widely regarded as the ...,othello complete amp unabridged,Visit Amazon's William Shakespeare Page,,0001048767,new used rental text humanities othello comp...
2,childrens literature fiction,,the secret garden,Frances Hodgson [illustrated by ruth sanderson...,$4.72,0001945424,childrens literature fiction the secret gar...
3,literature fiction literary,,ten little niggers,Visit Amazon's Agatha Christie Page,$50.63,0002318350,literature fiction literary ten little niggers
4,mystery thriller amp suspense thrillers amp s...,jan needle is a well known childrens writer hi...,dracula collins drama,Jan Needle,,0003302245,mystery thriller amp suspense thrillers amp s...


In [11]:
# book_metadata_df['brand'].value_counts()

In [12]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00


In [13]:
import torch
from transformers import BertTokenizer, BertModel

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model = model.to(device)

In [16]:
def generate_embeddings(sentences):
    # Tokenize the sentences and convert to tensors
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    inputs = inputs.to(device)

    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state
    attention_mask = inputs['attention_mask'].bool()
    sentence_embeddings = embeddings[attention_mask]

    return sentence_embeddings.cpu().numpy()

batch_size = 128
sentences = book_metadata_df['book_data'].tolist()
num_sentences = len(sentences)
embeddings_list = []

for i in range(0, num_sentences, batch_size):
    batch_sentences = sentences[i:i+batch_size]
    embeddings = generate_embeddings(batch_sentences)
    embeddings_list.append(embeddings)

all_embeddings = np.concatenate(embeddings_list, axis=0)

In [17]:
print(all_embeddings.shape)

(2665335, 768)


In [18]:
book_initial_embeddings = {item_id: list(emb) for item_id, emb in zip(book_metadata_df['asin'].tolist(), all_embeddings)}

# Export embeddings

In [19]:
#with open("/content/drive/MyDrive/Priyanka/book_initial_embeddings.json", "w") as f:
#    json.dump(book_initial_embeddings, f)

with open("/content/drive/MyDrive/Priyanka/book_initial_embeddings_bert.pkl", "wb") as f:
    pickle.dump(book_initial_embeddings, f)